In [289]:
import numpy as np
import pandas as pd

In [290]:
train_set = pd.read_csv('train_lyrics_1000.csv')
test_set = pd.read_csv('valid_lyrics_200.csv')

In [293]:
from sklearn.preprocessing import LabelEncoder
lenc = LabelEncoder();

lytrain = train_set['lyrics'].values  

Y_train = train_set['mood'].values

In [294]:
from string import punctuation
all_text = ''.join([c for c in lytrain if c not in punctuation])
lyrics = all_text.split('\n')

all_text = ' '.join(lyrics)
words = all_text.split()

In [295]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

lyrics_ints = []
for each in lyrics:
    lyrics_ints.append([vocab_to_int[word] for word in each.split()])

In [296]:
# Filter out that lyrics with 0 length
lyrics_ints = [each for each in lyrics_ints if len(each) > 0]

In [297]:
lyrics_lens = Counter([len(x) for x in lyrics_ints])
print("Zero-length lyrics: {}".format(lyrics_lens[0]))
print("Maximum lyrics length: {}".format(max(lyrics_lens)))

Zero-length lyrics: 0
Maximum lyrics length: 189


In [298]:
seq_len = 100
features = np.zeros((len(lytrain), seq_len), dtype=int)
for i, row in enumerate(lyrics_ints):
    #print("[%d] %s" % (i, row))
    features[i, -len(row):] = np.array(row)[:seq_len]

IndexError: index 999 is out of bounds for axis 0 with size 999

In [299]:
split_frac = 0.8
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = Y_train[:split_idx], Y_train[split_idx:]

test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(799, 100) 
Validation set: 	(100, 100) 
Test set: 		(100, 100)


In [305]:
lstm_size = 32
lstm_layers = 1
batch_size = 110
learning_rate = 0.001

In [306]:
import tensorflow as tf
n_words = len(vocab)

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [307]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 128

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [308]:
with graph.as_default():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [309]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed,initial_state=initial_state)

In [310]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [311]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [312]:
def get_batches(x, y, batch_size=110):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [313]:
epochs = 20

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%10==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, 'sentiment')

ValueError: invalid literal for int() with base 10: 'sad'

In [200]:
test_acc = []
tf.reset_default_graph()
saver = tf.train.import_meta_graph('sentiment.meta')
with tf.Session(graph=graph) as sess:
    
    saver.restore(sess, "sentiment")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from sentiment
Test accuracy: 0.436
